# CARS24 Web Scraping Project — Template Notebook

**Purpose:** Scrape used-car listings from Cars24 for a specified brand and location (Mumbai by default) and extract: kilometers driven, year, fuel type, transmission, and price.  
This notebook is a template: run it locally (with Chrome + ChromeDriver) and adapt selectors if Cars24 updates their layout.

**References / example pages used during planning**
- Example brand page (Mumbai): https://www.cars24.com/buy-used-hyundai-cars-mumbai/  
- Example model page (Mumbai): https://www.cars24.com/buy-used-honda-city-cars-mumbai/  

(If the site uses heavy JavaScript, the notebook uses Selenium to render pages before parsing.)


## 1) Research & Planning

- Cars24 listings are often rendered client-side; we therefore use Selenium to render pages and then parse the rendered HTML with BeautifulSoup.
- Common URL pattern (works for many brands):  
  `https://www.cars24.com/buy-used-{brand}-cars-{city}/`  
  Example: `https://www.cars24.com/buy-used-hyundai-cars-mumbai/`
- If no results are found for a city (e.g., Mumbai), the notebook will try a fallback list of cities (Mumbai, Pune, Bengaluru, Delhi, Hyderabad) and log any location changes.
- The notebook extracts these fields for each listing:
  - `brand`, `model` (when available), `year`, `kilometers_driven`, `fuel_type`, `transmission`, `price`, `listing_url`, `location`
- **Important:** If selectors fail due to site changes, update the CSS/XPath selectors in the `parse_listings` function. Example pages were inspected during planning.


In [ ]:
# Install requirements (run once in your environment)
# You can run these commands in your terminal or a Jupyter cell (with pip).
# NOTE: Don't run these here if you don't want to install packages in your environment.

# !pip install selenium beautifulsoup4 pandas tqdm webdriver-manager nbformat


In [ ]:
# Scraper template using Selenium + BeautifulSoup
# Configure brand (slug) and city. Brand slug usually is brand name in lowercase (e.g., 'hyundai', 'honda', 'maruti').
BRAND_SLUG = "hyundai"   # change this to your assigned brand
CITIES_TO_TRY = ["mumbai", "pune", "bangalore", "delhi", "hyderabad"]  # fallback order

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin
from tqdm import tqdm

def get_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    return driver

def build_brand_city_url(brand, city):
    # standard pattern used on Cars24
    return f"https://www.cars24.com/buy-used-{brand}-cars-{city}/"

def page_has_listings(soup):
    # heuristic: look for at least one listing card
    # Update selectors here if Cars24 changes structure
    return bool(soup.select("[data-test='carCard'], .car-listing, .card-listing, article"))

def parse_price(text):
    # clean price strings like '₹3.49 Lakh' or '₹3.49 L' or '₹3,49,000'
    import re
    t = text.replace(",", "").strip()
    # remove currency symbol
    t = re.sub(r"[^\d\.kKmM]", "", t)
    # basic conversions
    if t.lower().endswith('l') or 'lakh' in text.lower():
        num = float(re.findall(r"[\d\.]+", t)[0])
        return int(num * 100000)
    if t.lower().endswith('k'):
        num = float(re.findall(r"[\d\.]+", t)[0])
        return int(num * 1000)
    # fallback numbers
    nums = re.findall(r"[\d]+", t)
    if nums:
        return int(nums[0])
    return None

def parse_listings(soup, base_url):
    items = []
    # Try several likely card selectors — adapt if site updates
    card_selectors = ["[data-test='carCard']", ".car-listing", ".card-listing", "article", ".listingCard"]
    cards = []
    for sel in card_selectors:
        cards = soup.select(sel)
        if cards:
            break

    for c in cards:
        try:
            # Attempt to extract common pieces of information. These keys will be normalized later.
            title = c.get_text(" ", strip=True)
            # Price
            price_tag = c.select_one("[data-test='price'], .price, .rupee, .selling-price")
            price = price_tag.get_text(strip=True) if price_tag else ""
            price_clean = parse_price(price) if price else None

            # Year / kms / fuel / transmission: Many sites put them in a short-spec list
            spec_text = ""
            spec_el = c.select_one(".specs, .details, [data-test='carCardMeta']")
            if spec_el:
                spec_text = spec_el.get_text(" | ", strip=True)

            # Listing url
            a = c.select_one("a")
            listing_url = urljoin(base_url, a['href']) if a and a.has_attr('href') else base_url

            items.append({
                "raw_title": title,
                "price_raw": price,
                "price": price_clean,
                "spec_text": spec_text,
                "listing_url": listing_url
            })
        except Exception as e:
            # ignore card parse errors but log
            print("card parse error:", e)
    return items

def scrape_brand(brand, cities_list=CITIES_TO_TRY, headless=True, max_pages=3, wait=2):
    driver = get_driver(headless=headless)
    all_items = []
    chosen_city = None
    for city in cities_list:
        url = build_brand_city_url(brand, city)
        print("Trying:", url)
        driver.get(url)
        time.sleep(wait)  # let JS load; increase if needed
        soup = BeautifulSoup(driver.page_source, "html.parser")
        if page_has_listings(soup):
            chosen_city = city
            print(f"Found listings in {city} — proceeding.")
            # NOTE: pagination handling depends on the site. This template loads the first `max_pages` pages by clicking 'next' if available.
            # Simple loop to collect multiple pages (if buttons exist)
            page = 1
            while page <= max_pages:
                print("Scraping page", page)
                all_items.extend(parse_listings(soup, url))
                # Attempt to click next page (very generic)
                try:
                    next_btn = driver.find_element("xpath", "//button[contains(., 'Next') or contains(., '›') or contains(., '→')]")
                    if next_btn:
                        next_btn.click()
                        time.sleep(wait)
                        soup = BeautifulSoup(driver.page_source, "html.parser")
                        page += 1
                        continue
                except Exception:
                    break
            break
        else:
            print(f"No listings found for city {city}.")
    driver.quit()
    return all_items, chosen_city

# Example run (won't execute in this notebook environment unless you run it)
if __name__ == "__main__":
    # Note: set headless=False if you want to watch the browser while debugging.
    items, used_city = scrape_brand(BRAND_SLUG, headless=True, max_pages=2, wait=3)
    print(f"Scraped {len(items)} items from city: {used_city}")


In [ ]:
# Data cleaning and CSV export (works on the `items` list produced by the scraper)
import pandas as pd
import re

def clean_parsed_items(items):
    rows = []
    for it in items:
        title = it.get("raw_title","")
        spec = it.get("spec_text","")
        # Attempt to extract kms, year, fuel, transmission heuristically from spec/title
        # kms patterns examples: '42.44k km', '42,440 km', '42k km'
        kms = None
        m = re.search(r"([\d,\.]+)\s*k?m", spec, re.IGNORECASE)
        if not m:
            m = re.search(r"([\d,\.]+)\s*k?m", title, re.IGNORECASE)
        if m:
            kms_str = m.group(1).replace(",", "")
            # convert '42.44'k -> 42440 if decimal exists and 'k' present
            if 'k' in m.group(0).lower() or '.' in kms_str:
                try:
                    kms = int(float(kms_str) * 1000)
                except:
                    kms = int(float(kms_str))
            else:
                kms = int(float(kms_str))

        # year: look for 4-digit year like 2019
        year = None
        m = re.search(r"20\d{2}|19\d{2}", spec) or re.search(r"20\d{2}|19\d{2}", title)
        if m:
            year = int(m.group(0))

        # fuel type
        fuel = None
        ff = ["petrol","diesel","cng","electric","hybrid","lpg"]
        for f in ff:
            if f in spec.lower() or f in title.lower():
                fuel = f.capitalize()
                break

        # transmission
        trans = None
        if "auto" in spec.lower() or "automatic" in spec.lower() or "cv" in spec.lower():
            trans = "Automatic"
        elif "manual" in spec.lower() or "mt" in spec.lower():
            trans = "Manual"

        rows.append({
            "brand": BRAND_SLUG.capitalize(),
            "price_raw": it.get("price_raw"),
            "price": it.get("price"),
            "year": year,
            "kilometers": kms,
            "fuel": fuel,
            "transmission": trans,
            "listing_url": it.get("listing_url")
        })
    df = pd.DataFrame(rows)
    # Basic cleaning
    df = df.drop_duplicates(subset=["listing_url"])
    return df

# Example usage:
# df = clean_parsed_items(items)
# df.to_csv("cars24_{BRAND_SLUG}_{used_city}.csv", index=False)


## Notes / Troubleshooting

- Cars24 heavily uses JavaScript; if you get empty pages, increase `wait` or use `headless=False` to debug.
- If the HTML structure changes, open one listing in your browser, inspect the elements and update the `card_selectors` or the parsing logic in `parse_listings`.
- Always respect site ToS. For large-scale scraping, check robots.txt and consider contacting the site for an API or permission.
- The notebook logs which city was used. If Mumbai has no listings for the selected brand, the notebook tries fallback cities and documents that change in the `used_city` returned value.
